# Découverte de PVDAQ - Farm Solar Array - 2024

In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# On a récupéré le dataset PVDAQ pour le système Farm Solar Array pour l'année 2024
# https://openei.org/wiki/PVDAQ/Sites/Farm_Solar_Array

df_elec1 = pd.read_csv("pvdaq_data_2024/2107_electrical_data_2024.csv", parse_dates=['measured_on'])
df_env1  = pd.read_csv("pvdaq_data_2024/2107_environment_data_2024.csv", parse_dates=['measured_on'])
df_irr1  = pd.read_csv("pvdaq_data_2024/2107_irradiance_data_2024.csv", parse_dates=['measured_on'])
df_meter1 = pd.read_csv("pvdaq_data_2024/2107_meter_15m_data_2024.csv", parse_dates=['measured_on'])

df_elec2 = pd.read_csv("pvdaq_data_2017_a_2023/2107_electrical_data_v1.csv", parse_dates=['measured_on'])
df_env2  = pd.read_csv("pvdaq_data_2017_a_2023/2107_environment_data.csv", parse_dates=['measured_on'])
df_irr2  = pd.read_csv("pvdaq_data_2017_a_2023/2107_irradiance_data.csv", parse_dates=['measured_on'])
df_meter2 = pd.read_csv("pvdaq_data_2017_a_2023/2107_meter_15m_data.csv", parse_dates=['measured_on'])

# Concaténation
df_elec = pd.concat([df_elec2, df_elec1], ignore_index=True)
df_env = pd.concat([df_env2, df_env1], ignore_index=True)
df_irr = pd.concat([df_irr2, df_irr1], ignore_index=True)
df_meter = pd.concat([df_meter2, df_meter1], ignore_index=True)

#### Nettoyage des identifiants des capteurs à la fin des noms des colonnes pour plus de clarté.

In [3]:
def clean_column_name(col):
    # supprime _inv_XXXXX ou _o_XXXXX
    return re.sub(r"_(inv|o|meter)_[0-9]+$", "", col)

df_elec.columns = [clean_column_name(c) for c in df_elec.columns]
df_env.columns = [clean_column_name(c) for c in df_env.columns]
df_irr.columns = [clean_column_name(c) for c in df_irr.columns]
df_meter.columns = [clean_column_name(c) for c in df_meter.columns]

print(df_elec.columns)
print(df_env.columns)
print(df_irr.columns)
print(df_meter.columns)


Index(['measured_on', 'inv_01_dc_current', 'inv_01_dc_voltage',
       'inv_01_ac_current', 'inv_01_ac_voltage', 'inv_01_ac_power',
       'inv_02_dc_current', 'inv_02_dc_voltage', 'inv_02_ac_current',
       'inv_02_ac_voltage',
       ...
       'inv_23_dc_voltage', 'inv_23_ac_current', 'inv_23_ac_voltage',
       'inv_23_ac_power', 'inv_24_dc_current', 'inv_24_dc_voltage',
       'inv_24_ac_current', 'inv_24_ac_voltage', 'inv_24_ac_power',
       'inv_05_dc_voltage'],
      dtype='object', length=121)
Index(['measured_on', 'ambient_temperature', 'wind_speed', 'wind_direction'], dtype='object')
Index(['measured_on', 'poa_irradiance'], dtype='object')
Index(['measured_on', 'meter_revenue_grade_ac_output'], dtype='object')


#### On merge les 4 dataset ensemble

On convertit les colonnes "measured_on" en datetime ce qui permet un merge exact et on garde seulement les timestamps communs à tous les datasets.

In [4]:
for df in [df_elec, df_env, df_irr, df_meter]:
    df['measured_on'] = pd.to_datetime(df['measured_on'])

In [5]:
merged = df_elec.merge(df_env, on='measured_on', how='inner') \
            .merge(df_irr, on='measured_on', how='inner') \
            .merge(df_meter, on='measured_on', how='inner')

## **Exploration du dataset mergé**

In [6]:
merged.head()

,measured_on,inv_01_dc_current,inv_01_dc_voltage,inv_01_ac_current,inv_01_ac_voltage,inv_01_ac_power,inv_02_dc_current,inv_02_dc_voltage,inv_02_ac_current,inv_02_ac_voltage,...,inv_24_dc_voltage,inv_24_ac_current,inv_24_ac_voltage,inv_24_ac_power,inv_05_dc_voltage,ambient_temperature,wind_speed,wind_direction,poa_irradiance,meter_revenue_grade_ac_output
0,2017-12-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,38.8,1.2,156.0,0.0,0.0
1,2017-12-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,37.0,2.6,247.0,0.0,0.0
2,2017-12-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,37.0,2.6,247.0,0.0,0.0
3,2017-12-01 01:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,37.0,2.6,247.0,0.0,0.0
4,2017-12-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,37.5,1.9,265.0,0.0,0.0


In [7]:
print(merged.dtypes)

measured_on                      datetime64[ns]
inv_01_dc_current                       float64
inv_01_dc_voltage                       float64
inv_01_ac_current                       float64
inv_01_ac_voltage                       float64
                                      ...      
ambient_temperature                     float64
wind_speed                              float64
wind_direction                          float64
poa_irradiance                          float64
meter_revenue_grade_ac_output           float64
Length: 126, dtype: object


In [8]:
print(merged.shape)

(199711, 126)


In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199711 entries, 0 to 199710
Columns: 126 entries, measured_on to meter_revenue_grade_ac_output
dtypes: datetime64[ns](1), float64(125)
memory usage: 192.0 MB


In [10]:
merged.describe()

,measured_on,inv_01_dc_current,inv_01_dc_voltage,inv_01_ac_current,inv_01_ac_voltage,inv_01_ac_power,inv_02_dc_current,inv_02_dc_voltage,inv_02_ac_current,inv_02_ac_voltage,...,inv_24_dc_voltage,inv_24_ac_current,inv_24_ac_voltage,inv_24_ac_power,inv_05_dc_voltage,ambient_temperature,wind_speed,wind_direction,poa_irradiance,meter_revenue_grade_ac_output
count,199711,199711.000000,199711.000000,199711.000000,199711.00000,199711.000000,199711.000000,199711.000000,199711.000000,199711.000000,...,199711.000000,199711.000000,199711.000000,199711.000000,24391.000000,199580.000000,199695.000000,199703.000000,199711.000000,199711.000000
mean,2021-04-02 17:11:27.283624448,11.746963,374.509008,8.844126,160.33289,7.545081,11.040920,371.208704,8.339900,159.173965,...,381.964213,8.650573,163.534551,7.395053,421.243825,63.591876,5.701643,182.215397,268.612198,180.572732
min,2017-12-01 00:15:00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,23.400000,1.000000,0.000000,0.000000,0.000000
25%,2019-07-04 16:52:30,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,51.500000,2.900000,120.000000,0.000000,0.000000
50%,2021-02-28 10:45:00,1.200000,612.634000,1.339000,279.06700,0.315000,0.447000,598.655000,1.059000,279.632000,...,608.976000,1.420000,279.779000,0.110000,614.169000,62.300000,4.400000,153.000000,38.200000,17.280000
75%,2022-12-15 14:37:30,22.907500,682.601500,17.754000,287.11600,15.188000,20.892000,681.457000,16.327000,287.848000,...,687.603000,17.098000,286.894000,14.718500,672.739500,75.100000,7.300000,262.000000,546.600000,381.440000
max,2024-11-01 23:15:00,52.160000,909.840000,36.338000,310.67700,30.085000,54.844000,1750.755000,35.907000,310.436000,...,912.795000,35.848000,310.605000,30.089000,851.387000,115.600000,30.900000,360.000000,1400.000000,1783.680000
std,NaN,16.123177,334.581506,11.883779,142.14886,10.400228,15.801546,334.134676,11.564696,142.679477,...,335.311388,11.798319,141.548783,10.384771,314.551821,15.687559,4.013714,98.294213,348.407287,234.373358


Valeurs manquantes

In [11]:
missing = merged.isna().sum()
missing = missing[missing > 0]

print(missing)


inv_05_dc_voltage      175320
ambient_temperature       131
wind_speed                 16
wind_direction              8
dtype: int64


In [12]:
print("Plage des dates où les mesures ont été réalisées")
print(merged['measured_on'].min())
print(merged['measured_on'].max())


print("\nNombre de points par jours en moyenne")
daily_counts = merged['measured_on'].dt.date.value_counts().sort_index()
print(np.average(daily_counts))

Plage des dates où les mesures ont été réalisées
2017-12-01 00:15:00
2024-11-01 23:15:00

Nombre de points par jours en moyenne
83.4563309653155


In [13]:
cols_of_interest = [ 
    "measured_on",
    "poa_irradiance",
    "wind_speed",
    "wind_direction",
    "ambient_temperature",
    "meter_revenue_grade_ac_output",
]

colonnes_a_ajouter = [
"inv_01_dc_current","inv_01_dc_voltage","inv_01_ac_current","inv_01_ac_voltage","inv_01_ac_power",
"inv_02_dc_current","inv_02_dc_voltage","inv_02_ac_current","inv_02_ac_voltage","inv_02_ac_power",
"inv_03_dc_current","inv_03_dc_voltage","inv_03_ac_current","inv_03_ac_voltage","inv_03_ac_power",
"inv_04_dc_current","inv_04_dc_voltage","inv_04_ac_current","inv_04_ac_voltage","inv_04_ac_power",
"inv_05_dc_current","inv_05_dc_voltage","inv_05_ac_current","inv_05_ac_voltage","inv_05_ac_power",
"inv_06_dc_current","inv_06_dc_voltage","inv_06_ac_current","inv_06_ac_voltage","inv_06_ac_power",
"inv_07_dc_current","inv_07_dc_voltage","inv_07_ac_current","inv_07_ac_voltage","inv_07_ac_power",
"inv_08_dc_current","inv_08_dc_voltage","inv_08_ac_current","inv_08_ac_voltage","inv_08_ac_power",
"inv_09_dc_current","inv_09_dc_voltage","inv_09_ac_current","inv_09_ac_voltage","inv_09_ac_power",
"inv_10_dc_current","inv_10_dc_voltage","inv_10_ac_current","inv_10_ac_voltage","inv_10_ac_power",
"inv_11_dc_current","inv_11_dc_voltage","inv_11_ac_current","inv_11_ac_voltage","inv_11_ac_power",
"inv_12_dc_current","inv_12_dc_voltage","inv_12_ac_current","inv_12_ac_voltage","inv_12_ac_power",
"inv_13_dc_current","inv_13_dc_voltage","inv_13_ac_current","inv_13_ac_voltage","inv_13_ac_power",
"inv_14_dc_current","inv_14_dc_voltage","inv_14_ac_current","inv_14_ac_voltage","inv_14_ac_power",
"inv_15_dc_current","inv_15_dc_voltage","inv_15_ac_current","inv_15_ac_voltage","inv_15_ac_power",
"inv_16_dc_current","inv_16_dc_voltage","inv_16_ac_current","inv_16_ac_voltage","inv_16_ac_power",
"inv_17_dc_current","inv_17_dc_voltage","inv_17_ac_current","inv_17_ac_voltage","inv_17_ac_power",
"inv_18_dc_current","inv_18_dc_voltage","inv_18_ac_current","inv_18_ac_voltage","inv_18_ac_power",
"inv_19_dc_current","inv_19_dc_voltage","inv_19_ac_current","inv_19_ac_voltage","inv_19_ac_power",
"inv_20_dc_current","inv_20_dc_voltage","inv_20_ac_current","inv_20_ac_voltage","inv_20_ac_power",
"inv_21_dc_current","inv_21_dc_voltage","inv_21_ac_current","inv_21_ac_voltage","inv_21_ac_power",
"inv_22_dc_current","inv_22_dc_voltage","inv_22_ac_current","inv_22_ac_voltage","inv_22_ac_power",
"inv_23_dc_current","inv_23_dc_voltage","inv_23_ac_current","inv_23_ac_voltage","inv_23_ac_power",
"inv_24_dc_current","inv_24_dc_voltage","inv_24_ac_current","inv_24_ac_voltage","inv_24_ac_power"
]


cols_of_interest += colonnes_a_ajouter

df = merged[cols_of_interest].copy()
df


,measured_on,poa_irradiance,wind_speed,wind_direction,ambient_temperature,meter_revenue_grade_ac_output,inv_01_dc_current,inv_01_dc_voltage,inv_01_ac_current,inv_01_ac_voltage,...,inv_23_dc_current,inv_23_dc_voltage,inv_23_ac_current,inv_23_ac_voltage,inv_23_ac_power,inv_24_dc_current,inv_24_dc_voltage,inv_24_ac_current,inv_24_ac_voltage,inv_24_ac_power
0,2017-12-01 00:15:00,0.0,1.2,156.0,38.8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-12-01 01:00:00,0.0,2.6,247.0,37.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-12-01 01:30:00,0.0,2.6,247.0,37.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-12-01 01:45:00,0.0,2.6,247.0,37.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-12-01 02:00:00,0.0,1.9,265.0,37.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199706,2024-11-01 21:45:00,0.0,13.1,109.0,55.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199707,2024-11-01 22:15:00,0.0,12.9,107.0,54.9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199708,2024-11-01 22:45:00,0.0,12.9,107.0,54.9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199709,2024-11-01 23:00:00,0.0,11.7,99.0,54.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
